In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from normal_normal_model import get_samples
from covariance_CI_test import *
import pickle
from typing import List
import time

In [ ]:
# N = 500
# M = 100
# mu_X = 0
# mu_epsilon = 0
# sigma_sq_X = 1.5**2
# sigma_sq_1 = 1.1**2
# sigma_sq_2 = 0.6**2

# r = 450
# s = 485

In [ ]:
num_runs = 47
num_covariate_samples = 47
num_tests = 300

In [ ]:
def print_covariance_CI_test_results(covariance_CI_tests: List[CovarianceCITest]) -> None:
    if covariance_CI_tests is None or len(covariance_CI_tests) == 0:
        print("There is nothing in the provided test collection.")
    
    within_CI = [test.theoretical_quantity_in_sample_CI()
                 for test in covariance_CI_tests]
    
    print(covariance_CI_tests[0].get_test_name() + 
          ": {}/{} ({}%) "
          .format(np.sum(within_CI), len(within_CI), 
                  np.round(100.0*np.sum(within_CI)/len(within_CI), 2)) +
          "of the tests have the theoretical quantity within the CI.")

In [ ]:
def save_test_collection(test_collection: List[CovarianceCITest]) -> None:
    if test_collection is None or len(test_collection) == 0:
        return
    file_name = ("../output/" + str(test_collection[0].__class__.__name__) + 
                 "_" + str(int(time.time())) + ".pickle")
    pickle_file = open(file_name, 'wb')
    pickle.dump(test_collection, pickle_file)
    
    print("The test collection is saved at " + file_name)

In [ ]:
def get_params() -> Dict[str, float]:
    """
    Randomly generate a set of parameters from a realistic param space
    """
    params = {
        'N': int(10 ** np.random.uniform(1.0, 3.5)),
        'mu_X': np.random.uniform(-10, 10),
        'mu_epsilon': np.random.uniform(-10, 10),
        'sigma_sq_X': np.random.uniform(0.3, 10) ** 2,
        'sigma_sq_1': np.random.uniform(0.3, 10) ** 2
    }
    params['M'] = max(int(params['N'] * np.random.uniform(0.01, 0.66)), 1)
    params['sigma_sq_2'] = (
        max((np.sqrt(params['sigma_sq_1']) * np.random.uniform(0.1, 0.99)) ** 2, 
            0.2 ** 2))
    params['r'] = (
        int(np.random.uniform(params['N'] - params['M'] + 1, params['N'] + 1)))
    params['s'] = (
        int(np.random.uniform(params['N'] - params['M'] + 1, params['N'] + 1)))
        
    return params

In [ ]:
cov_Yr_Zs_tests = []
cov_Yr_Ys_second_order_tests = []
cov_XIr_XJs_tests = []
cov_V1_V2_tests = []
cov_Yr_Ys_tests = []

cov_test_collections = [
    cov_Yr_Ys_tests, cov_Yr_Ys_second_order_tests,
    cov_Yr_Zs_tests, cov_XIr_XJs_tests, cov_V1_V2_tests]

for num_test in range(0, num_tests):
    try:
        # Sample the parameters from a realistic parameter space
        params = get_params()
        N = params['N']
        M = params['M']
        mu_X = params['mu_X']
        mu_epsilon = params['mu_epsilon']
        sigma_sq_X = params['sigma_sq_X']
        sigma_sq_1 = params['sigma_sq_1']
        sigma_sq_2 = params['sigma_sq_2']
        r = params['r']
        s = params['s']
        
        # Prepare a test
        cov_Yr_Ys_test = CovYrYsCITest(r, s, N, sigma_sq_X, sigma_sq_1=sigma_sq_1)
        cov_Yr_Ys_second_order_test = CovYrYsSecondOrderCITest(r, s, N, sigma_sq_X, sigma_sq_1=sigma_sq_1)
        cov_Yr_Zs_test = CovYrZsCITest(r, s, sigma_sq_X, sigma_sq_1, sigma_sq_2, N)
        cov_XIr_XJs_test = CovXIrXJsCITest(r, s, sigma_sq_X, sigma_sq_1, sigma_sq_2, N)
        cov_V1_V2_test = CovV1V2CITest(sigma_sq_X, sigma_sq_1, sigma_sq_2, N, M)
        
        # Arrange the covariance tests IN THE SAME ORDER as that in the collections
        cov_tests = [cov_Yr_Ys_test, cov_Yr_Ys_second_order_test, 
                     cov_Yr_Zs_test, cov_XIr_XJs_test, cov_V1_V2_test]

        for num_covariate_sample in range(0, num_covariate_samples):
            print("Test {}/{} (N={}, M={}): calculating sample {}/{}...      "
                  .format(num_test+1, num_tests, N, M,
                          num_covariate_sample+1, num_covariate_samples), 
                  end='\r')

            # Generate samples from multiple runs to obtain one
            # covariate sample for each pair of r.v. we are intersted in
            samples = get_samples(
                num_runs, N, M, mu_X, mu_epsilon, 
                sigma_sq_X, sigma_sq_1, sigma_sq_2,
                verbose=False, r=r, s=s)

            for test in cov_tests:
                test.add_sample(samples)
        
        # Once a test is completed (i.e. we collected enough covariate
        # samples), we added them to the corresponding, existing
        # collection of tests
        for (cov_test, cov_test_collection) in \
            [(cov_tests[i], cov_test_collections[i]) for i in range(0, len(cov_tests))]:
            cov_test_collection.append(cov_test)
        
    except KeyboardInterrupt:
        # Breaking the for loop is sufficient, as we want to
        # analyse the results accumulated so far
        break

for cov_test_collection in cov_test_collections:
    print_covariance_CI_test_results(cov_test_collection)
    save_test_collection(cov_test_collection)

# TODO: Save the tests as a pickle for future reference. https://www.thoughtco.com/using-pickle-to-save-objects-2813661
# TODO: Convert this to a callable program https://stackabuse.com/command-line-arguments-in-python/

In [ ]:
for cov_test_collection in cov_test_collections:
    print_covariance_CI_test_results(cov_test_collection)
    save_test_collection(cov_test_collection)

In [ ]:
print_covariance_CI_test_results(cov_Yr_Zs_tests)
print_covariance_CI_test_results(cov_XIr_XJs_tests)
print_covariance_CI_test_results(cov_V1_V2_tests)

In [ ]:
print(np.sum([test.theoretical_quantity_above_sample_CI() for test in cov_Yr_Ys_tests]))
print(np.sum([test.theoretical_quantity_below_sample_CI() for test in cov_Yr_Ys_tests]))
print(np.sum([test.theoretical_quantity_above_sample_CI() for test in cov_Yr_Ys_second_order_tests]))
print(np.sum([test.theoretical_quantity_below_sample_CI() for test in cov_Yr_Ys_second_order_tests]))

In [ ]:
for test in cov_Yr_Ys_tests:
    if not test.theoretical_quantity_in_sample_CI():
        print(test.samples)
#         plt.gca()
#         plt.hist(test.samples)
#         plt.axvline(test.theoretical_quantity())
#         print(test.get_sample_CI(), test.theoretical_quantity()) 

In [ ]:
cov_Yr_Ys_tests[0].samples

In [ ]:
print([test.theoretical_quantity() for test in cov_Yr_Ys_tests])
print([test.theoretical_quantity() for test in cov_Yr_Zs_second_order_tests])